In [126]:
import pandas as pd
import requests
from requests.exceptions import HTTPError


data_url = "https://www.fhwa.dot.gov/policyinformation/travel_monitoring/18novtvt/18novtvt.xls"
data = pd.read_excel(data_url, "Page 6", skiprows=7)
data = data.rename(columns={"Unnamed: 0": "State"})


In [106]:
years = ["16", "17", "18"]
months = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]
excel_files=[]

for months in months:
    for year in years:
        excel_files.append(year + months)
        

In [116]:
month_dfs = []

for file in excel_files:
    
    try:
        data_url = f"https://www.fhwa.dot.gov/policyinformation/travel_monitoring/{file}tvt/{file}tvt.xls"
        data = pd.read_excel(data_url, "Page 6", skiprows=7)
        data = data.rename(columns={"Unnamed: 0": "State"})
        data = data[["State", "STATIONS", "CMILES"]]
        data = data.dropna()
        data = data[data["State"] != "TOTALS"]
        data = data.rename(columns={"STATIONS": f"{file} Stations"})
        data = data.rename(columns={"CMILES": f"{file} Miles"})
        data = data.set_index("State")
        month_dfs.append(data)
    
    except:
        print(f"{file} not found ....skipping")
        continue
    
state_vmt = pd.DataFrame()
state_vmt = pd.concat(month_dfs, axis=1, sort=True)
state_vmt = state_vmt.dropna()
state_vmt.head()

18oct not found ....skipping
18dec not found ....skipping


,16jan Stations,16jan Miles,17jan Stations,17jan Miles,18jan Stations,18jan Miles,16feb Stations,16feb Miles,17feb Stations,17feb Miles,...,16nov Stations,16nov Miles,17nov Stations,17nov Miles,18nov Stations,18nov Miles,16dec Stations,16dec Miles,17dec Stations,17dec Miles
State,,,,,,,,,,,,,,,,,,,,,
Alabama,87,5146,103,5087,81,5101,103,5153,109,5126,...,105,5395,81,5392,146,5349,108,5601,79,5564
Alaska,84,362,86,354,97,355,85,337,92,326,...,66,367,97,366,92,373,92,361,98,365
Arizona,153,5470,0,5633,150,5717,132,5494,115,5496,...,175,5170,132,5318,138,5390,179,6037,123,6141
Arkansas,29,3048,31,3040,39,3062,23,2745,35,2364,...,31,2676,36,2806,37,2606,33,3017,38,3167
California,191,24667,167,23997,156,25629,165,25229,54,24050,...,156,34354,160,33156,96,32119,175,33859,166,32638


In [117]:
state_vmt

,16jan Stations,16jan Miles,17jan Stations,17jan Miles,18jan Stations,18jan Miles,16feb Stations,16feb Miles,17feb Stations,17feb Miles,...,16nov Stations,16nov Miles,17nov Stations,17nov Miles,18nov Stations,18nov Miles,16dec Stations,16dec Miles,17dec Stations,17dec Miles
State,,,,,,,,,,,,,,,,,,,,,
Alabama,87,5146,103,5087,81,5101,103,5153,109,5126,...,105,5395,81,5392,146,5349,108,5601,79,5564
Alaska,84,362,86,354,97,355,85,337,92,326,...,66,367,97,366,92,373,92,361,98,365
Arizona,153,5470,0,5633,150,5717,132,5494,115,5496,...,175,5170,132,5318,138,5390,179,6037,123,6141
Arkansas,29,3048,31,3040,39,3062,23,2745,35,2364,...,31,2676,36,2806,37,2606,33,3017,38,3167
California,191,24667,167,23997,156,25629,165,25229,54,24050,...,156,34354,160,33156,96,32119,175,33859,166,32638
Colorado,98,4401,96,4429,80,4740,88,3861,77,3985,...,85,3992,80,4054,102,3869,92,4393,80,4479
Connecticut,32,2417,34,2456,0,2273,31,2265,35,2274,...,33,2692,21,2746,11,2656,33,2527,24,2513
Delaware,70,694,67,711,55,719,65,699,60,720,...,48,753,52,749,0,744,54,768,55,746
District of Columbia,2,285,2,306,3,362,1,224,3,206,...,3,277,3,288,2,292,3,298,3,310


In [129]:
import pymysql
pymysql.install_as_MySQLdb()

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Integer, String, Float

from sqlalchemy import create_engine

dbuser = 'root'
dbpassword = 'ro-Cha7t'
dbhost = 'localhost'
dbport = '3306'
dbname= 'ETL_db'

engine = create_engine(f"mysql://{dbuser}:{dbpassword}@{dbhost}:{dbport}/{dbname}")
Base.metadata.create_all(engine)

In [128]:

state_vmt.to_sql('vmt', engine, if_exists="replace")

InternalError: (pymysql.err.InternalError) (1170, "BLOB/TEXT column 'State' used in key specification without a key length") [SQL: 'CREATE INDEX `ix_vmt_State` ON vmt (`State`)'] (Background on this error at: http://sqlalche.me/e/2j85)